# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [4]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()


## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [5]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [6]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [7]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [8]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist
0,232,Achtung Baby,U2
1,233,All That You Can't Leave Behind,U2
2,234,B-Sides 1980-1990,U2
3,235,How To Dismantle An Atomic Bomb,U2
4,255,Instant Karma: The Amnesty International Campa...,U2
5,236,Pop,U2
6,237,Rattle And Hum,U2
7,238,The Best Of 1980-1990,U2
8,239,War,U2
9,240,Zooropa,U2


In [9]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [11]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

#### Les avantages

Voici les avantages de l'utilisation des vues dans SQL:

- Si nous devons conserver des informations sensibles en fournissant un accès limité aux utilisateurs, des vues sont utilisées à cette fin. Les vues sont utilisées pour afficher uniquement les données requises aux utilisateurs en protégeant les données sensibles.

- Comme une vue de base de données est associée à de nombreuses tables sur lesquelles la vue est créée, elle simplifie la complexité de la requête.

- La vue est utilisée pour cacher aux utilisateurs finaux la complexité des tables sous-jacentes utilisées dans une base de données.

- Les vues sont utiles en cas de re-conception de la base de données afin de ne pas affecter les autres applications utilisant la même base de données.

- Les données des colonnes calculées peuvent être calculées très facilement lorsque nous interrogeons les données de la vue, car les vues permettent les colonnes calculées.

#### Les inconvénients

Malgré les nombreux avantages qu'offre la vue, elle présente toujours certains inconvénients, comme indiqué ci-dessous:

- L'un des principaux inconvénients de l'utilisation de la vue apparaît dans l'image lorsque nous modifions fréquemment les structures de table sur lesquelles la vue est créée. Ainsi, lorsque les structures de table sont modifiées, la vue doit également être modifiée.

- En outre, l'utilisation de la vue ralentit les performances des requêtes.

#### Conclusion

Les vues sont largement utilisées pour leurs nombreux avantages. Elles ajoutent une couche de sécurité supplémentaire à la base de données, ce qui est très essentiel pour tout système de gestion de base de données relationnelle.
Les vues sont flexibles en cas d'exposition des données aux utilisateurs finaux en affichant uniquement les données nécessaires, par exemple en utilisant des vues en lecture seule pour limiter les privilèges aux utilisateurs.
Mais les vues peuvent également être désavantageuses si les structures de table sous-jacentes changent beaucoup / fréquemment, augmentant ainsi la complexité du changement des vues en fonction des structures de table.
Ainsi, avec de nombreux avantages dans l'image, les vues sont un peu moins recommandées lorsque les performances de la requête de données sont vitales pour l'entreprise. Il dépend de nous de choisir l'utilisation de la vue dans notre base de données en validant correctement l'exigence métier afin de tirer plus d'avantages des vues afin d'augmenter les performances du système.

[Source](https://fr.photo-555.com/5751867-sql-views)

---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

In [12]:
# creation vue albums
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist,
        'Nb de pistes',
        'Durée totale de l''album (min.)',
        'Prix de l''album')
    AS
        SELECT al.AlbumId,
               al.Title,
               ar.Name AS Artist,
               COUNT(t.TrackId) AS 'Nb de pistes',
               ROUND(SUM(t.Milliseconds)/60000., 2) AS 'Durée Totale de l''album (min.)',
               SUM(t.UnitPrice) AS 'Prix de l'' album'
        FROM albums al
         INNER JOIN artists ar ON al.ArtistId = ar.ArtistId
            LEFT JOIN tracks t ON al.AlbumId = t.AlbumId
        GROUP BY Title    
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

In [13]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist,Nb de pistes,Durée totale de l'album (min.),Prix de l'album
0,1,For Those About To Rock We Salute You,AC/DC,10,40.01,9.90
1,4,Let There Be Rock,AC/DC,8,40.89,7.92
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1,3.30,0.99
3,267,Worlds,Aaron Goldberg,1,4.45,0.99
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2,7.75,1.98


### Vue 2: caractéristiques détaillées des pistes
Créer et exploiter une nouvelle vue v_tracks reprenant pour chaque piste:  
- TrackId
- Name
- Genre
- Album
- Artist
- Composer
- Temps (en seconde)
- MediaType

In [14]:
# creation vue albums avec artistes
creation_vue_tracks = """
    CREATE VIEW v_tracks (
        TrackId,
        Name,
        Genre,
        Album,
        Artist,
        Composer,
        'Temps (en secondes)',
        MediaType)
    AS
        SELECT
	t.TrackId,
	t.Name,
	g.Name AS Genre,
	al.Title AS Album,
	ar.Name AS Artist,
	t.Composer AS Composer,
	(t.Milliseconds)/1000 AS 'Temps (en seconde)',
	mt.Name AS MediaType
FROM
	tracks t
LEFT JOIN albums al ON
	al.AlbumId = t.AlbumId
LEFT JOIN artists ar ON
	al.ArtistId = ar.ArtistId
LEFT JOIN genres g ON
	g.GenreId = t.GenreId 
LEFT JOIN media_types mt ON
	mt.MediaTypeId = t.MediaTypeId
ORDER BY
	Artist;
"""

executer_requete(creation_vue_tracks, conn)

In [15]:
# Utilisation de la vue v_tracks
albums = pd.read_sql_query("SELECT * FROM v_tracks LIMIT 25;", conn)
albums.head(25)

,TrackId,Name,Genre,Album,Artist,Composer,Temps (en secondes),MediaType
0,1,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",343,MPEG audio file
1,6,Put The Finger On You,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",205,MPEG audio file
2,7,Let's Get It Up,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",233,MPEG audio file
3,8,Inject The Venom,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",210,MPEG audio file
4,9,Snowballed,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",203,MPEG audio file
5,10,Evil Walks,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",263,MPEG audio file
6,11,C.O.D.,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",199,MPEG audio file
7,12,Breaking The Rules,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",263,MPEG audio file
8,13,Night Of The Long Knives,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",205,MPEG audio file
9,14,Spellbound,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",270,MPEG audio file


### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

In [ ]:
# creation vue augemntée invoices
creation_vue_invoices = """
    CREATE VIEW v_invoices (
        AlbumId,
        Title,
        Artist,
        'Nb de pistes',
        'Durée totale de l''album (min.)',
        'Prix de l''album')
    AS
        SELECT al.AlbumId,
               al.Title,
               ar.Name AS Artist,
               COUNT(t.TrackId) AS 'Nb de pistes',
               ROUND(SUM(t.Milliseconds)/60000., 2) AS 'Durée Totale de l''album (min.)',
               SUM(t.UnitPrice) AS 'Prix de l'' album'
        FROM albums al
         INNER JOIN artists ar ON al.ArtistId = ar.ArtistId
            LEFT JOIN tracks t ON al.AlbumId = t.AlbumId
        GROUP BY Title    
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  